# 一、预训练基础模块（独立训练）
1. 自动编码器（VAE）的训练
目标：学习图像与潜在空间（Latent Space）的双向映射，实现高效的 “感知压缩”（将高分辨率图像压缩为低维潜在表示，再从潜在表示重建图像）。
输入数据：大量无标签图像（无需图文对），利用非成对图像数据训练。
训练过程：
编码器（Encoder）将输入图像（如 512x512 RGB 图像）压缩为低维潜在表示（如 4x64x64，通道数为 4），输出 8 通道特征（前 4 通道为均值 μ，后 4 通道为 log 方差 log (σ²)）。
解码器（Decoder）将潜在表示还原为像素级图像。
损失函数：采用 KL 散度（约束潜在空间分布接近标准高斯分布）和感知损失（LPIPS，确保重建图像的视觉质量）。
输出：训练完成的编码器（用于将图像转换为潜在表示）和解码器（用于从潜在表示生成图像）。
2. 文本编码器的预训练
目标：将文本提示（Prompt）转换为数值化的语义向量，建立文本与图像语义的关联。
输入数据：4 亿对图文数据（image-text pairs），通过对比学习训练。
训练过程：
采用 CLIP（或 Stable Diffusion v2 的 OpenCLIP）作为文本编码器，其核心是让 “成对的文本和图像向量距离更近，非成对的距离更远”。
输出为 77 个 token 的嵌入向量（每个向量 768 维），捕捉文本深层语义。
特点：文本编码器通常为预训练模型，在 Stable Diffusion 训练中直接冻结使用，不参与后续微调。
# 二、扩散模型（UNet + 调度器）的训练
1. 数据准备
采用高质量图文对数据（如 COCO Captions、NoCaps 等，总量约 13M），用于学习文本条件与图像潜在表示的关联。
2. 核心目标
训练 UNet 模型在潜在空间中学习 “从带噪潜在表示中预测并去除噪声” 的能力，并结合文本条件，使生成的潜在表示与文本描述对齐。
3. 训练过程
## 步骤 1：前向加噪过程
对 VAE 输出的潜在表示（4 通道）进行逐步加噪，由调度器（如 DDPM）控制噪声强度：加噪过程从无噪逐步到纯噪声，确保噪声添加的 “伪随机性”（给定初始种子可复现）。
## 步骤 2：反向去噪训练（UNet 训练核心）
训练 UNet 学习从带噪潜在表示中预测噪声，输入包括：
带噪潜在表示
时间步 t（表示当前加噪步骤，通过时间嵌入模块转换为高维向量）；
文本嵌入（由预训练文本编码器生成的 768 维语义向量）。
具体过程：
1. UNet 通过交叉注意力（Cross-Attention）融合文本嵌入与潜在表示特征，捕捉文本与图像的语义关联。
2. 预测当前步骤的噪声，损失函数为预测噪声与真实噪声的 MSE（均方误差）
3. 通过残差连接（ResnetBlock2D）和跳跃连接（保留多尺度特征）增强模型对细节的捕捉能力。
## 步骤 3：调度器协同优化
调度器（如 DDIM）与 UNet 协同训练，控制去噪步数和噪声强度，平衡生成速度与质量（默认 50-100 步），确保反向去噪过程的稳定性。
# 三、模块协同与微调（可选）
训练完成后，各模块通过 “潜在空间” 协同工作：
文本编码器输出的语义向量作为 UNet 的条件输入；
UNet 在潜在空间中根据文本条件去噪，生成与文本匹配的潜在表示；
解码器将潜在表示转换为最终图像。
针对特定任务（如超分辨率、图像修复），可在预训练基础上用任务专属数据微调 UNet，增强模块对特定场景的适应性。